In [1]:
from model import PhiClipMLLM
import torch

c:\Users\sunny\.conda\envs\condapy12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We first try out the phase 1 checkpoint.

In [2]:
mllm = PhiClipMLLM.from_pretrained(r"C:\Users\sunny\Desktop\mllm\checkpoint_dir\checkpoint-14786\model_checkpoint.pt", dtype=torch.bfloat16)

c:\Users\sunny\Desktop\projects\PhiClipMLLM\model.py:462: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_file, map_location="cuda")


Checkpoint loaded
Vision model loaded
Text model loaded
Vision adapter loaded


In [5]:
from model import generate
text = generate(mllm, ["Describe this image <|placeholder1|> <|placeholder2|>"], images=[r".\test_image\dune.jpg"])
print(text)

c:\Users\sunny\.conda\envs\condapy12\Lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\sunny\.conda\envs\condapy12\Lib\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


["A dog sitting on a table with his head in front of the ground looking at an animal's back legs and tail upward facing camera\ndog behind him. \u200b face down"]


Now  we try phase 2 checkpoint

In [2]:
mllm = PhiClipMLLM.from_pretrained(r"C:\Users\sunny\Desktop\mllm\checkpoint_dir\checkpoint-500\model_checkpoint.pt", dtype=torch.bfloat16)

c:\Users\sunny\Desktop\projects\PhiClipMLLM\model.py:462: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_file, map_location="cuda")


Checkpoint loaded
Vision model loaded
Text model loaded
Vision adapter loaded


In [3]:
from model import generate
text = generate(mllm, ["Describe this image <|placeholder1|> <|placeholder2|>"], images=[r".\test_image\dune.jpg"])
print(text)

c:\Users\sunny\.conda\envs\condapy12\Lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\sunny\.conda\envs\condapy12\Lib\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


['A dog sitting on a table with its head in front legs up and tail laying down, face looking at the ground facing side of back feet rests pawses 2 black']


Berhavior of batched computation

In [11]:
input_texts = ["Here is an image: <|placeholder1|>. what is happening <|placeholder2|> the happening is what",
            ]
input_images = torch.ones(1, 3, 224, 224) 

out = mllm.__call__(
    input_texts = input_texts,
    input_images = input_images,
    labels = input_texts
)
out

{'logits': tensor([[[22.0000, 21.7500, 23.2500,  ..., 17.6250, 17.6250, 17.6250],
          [28.7500, 30.8750, 33.5000,  ..., 26.7500, 26.7500, 26.7500],
          [28.2500, 26.5000, 30.5000,  ..., 23.3750, 23.3750, 23.3750],
          ...,
          [33.2500, 36.5000, 35.0000,  ..., 28.6250, 28.6250, 28.6250],
          [35.5000, 36.7500, 35.5000,  ..., 30.5000, 30.5000, 30.5000],
          [34.2500, 35.5000, 35.5000,  ..., 29.8750, 29.8750, 29.8750]]],
        device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>),
 'loss': tensor(5.5625, device='cuda:0', dtype=torch.bfloat16,
        grad_fn=<NllLossBackward0>)}

In [12]:
input_texts = ["Here is an image: <|placeholder1|>. what is happening <|placeholder2|> the happening is what",
               "What is happening: <|placeholder1|>.  <|placeholder2|> What the hell",
            ]
input_images = torch.ones(2, 3, 224, 224) 

out = mllm.__call__(
    input_texts = input_texts,
    input_images = input_images,
    labels = input_texts
)
out["loss"]

tensor(5.2188, device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<NllLossBackward0>)